In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, roc_auc_score, brier_score_loss,
    precision_recall_fscore_support, confusion_matrix
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.linear_model import GammaRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import root_mean_squared_error
from scipy.stats import randint, uniform


# 1st Model: Frequency Model (probability of claim vs no claim)

# load the dataset from the csv file using pandas to read it
df = pd.read_csv("../data/h251.csv")

# chose target variable to be total expenditures (total amount of money a person spent on healthcare in 2023)
target = "TOTEXP23"

# chose predictors to train the model on
# predictors are age, sex, race, region, cancer, high blood pressure, asthma, coronary heart disease, insurance coverage
predictors = [
    # demographics
    "AGE23X", "SEX", "RACETHX", "REGION23", "MARRY23X", 
    # socioeconomic
    "POVCAT23", "INSCOV23", 
    # illnesses / chronic conditions
    "CANCERDX", "HIBPDX", "ASTHDX", "CHDDX", "DIABDX_M18", 
    # lifestyle
    "ADSMOK42"
]

# concatenate each line of predictors with the target value and drop any missing values
df = df[predictors + [target]].dropna()

# changes the 2 to be 0 so that it works as binary values for the model
for c in ["CANCERDX","HIBPDX","ASTHDX","CHDDX", "DIABDX_M18"]:
    if df[c].dtype != 'float' and df[c].dtype != 'int':
        pass
    df[c] = df[c].map({1:1, 2:0}).fillna(df[c])

# assign X to be the set of predictors
X = df[predictors]
# assign y to be the target; converts the value into a binary so that if the total expenditures is >= 1, 
# it's 1, otherwise it's 0
y = (df["TOTEXP23"] > 0).astype(int)

# splits dataset to train 80% and test 20%, set random_state to an integer so that it controls the randomness of how it splits the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=23)

categorical_cols = ["SEX", "RACETHX", "REGION23", "MARRY23X", "POVCAT23", "INSCOV23", "ADSMOK42"]
numerical_cols = ["AGE23X", "CANCERDX", "HIBPDX", "ASTHDX", "CHDDX", "DIABDX_M18"]

# data preprocessing since LogisticRegression will mistake the categorical values for numerical values
# use OneHotEncoder to transform each category into a binary column 
# use StandardScaler scale the numerical values
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop=None), categorical_cols),
        ("num", StandardScaler(), numerical_cols),
    ]
)

preprocess.fit(X_train)

# using a pipeline so that preprocessing happens before fitting the data to the LogisticRegression model
base_clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=5000, class_weight="balanced", solver="liblinear"))
])

param_grid = {
    "model__C": [0.1, 0.5, 1.0, 2.0, 5.0],
    "model__penalty": ["l1", "l2"]
}
clf = GridSearchCV(base_clf, param_grid, cv=5, scoring="roc_auc", n_jobs=-1)

clf.fit(X_train, y_train)

# predict probabilities and classify with threshold 0.3
proba = clf.predict_proba(X_test)[:,1]
y_hat = (proba >= 0.3).astype(int)

# calculates metrics for logistic regression
acc  = accuracy_score(y_test, y_hat)
auc  = roc_auc_score(y_test, proba)
brier = brier_score_loss(y_test, proba)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_hat, average="binary")
cm = confusion_matrix(y_test, y_hat)

print(f"Logistic Regression Model Stats")
print(f"Class balance (positive rate): {y.mean():.3f}")
print(f"Accuracy    : {acc:.3f}")
print(f"ROC-AUC     : {auc:.3f}")
print(f"Brier score : {brier:.3f}")
print(f"Precision   : {prec:.3f}  Recall: {rec:.3f}  F1: {f1:.3f}")
print("Confusion matrix:\n", cm)


Logistic Regression Model Stats
Class balance (positive rate): 0.860
Accuracy    : 0.848
ROC-AUC     : 0.828
Brier score : 0.177
Precision   : 0.912  Recall: 0.910  F1: 0.911
Confusion matrix:
 [[ 255  285]
 [ 291 2953]]


In [13]:
# HistGradientBoostingClassifier Model
improved_clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", HistGradientBoostingClassifier(
        learning_rate=0.1,
        max_depth=None,
        max_iter=300,
        l2_regularization=0.0,
        early_stopping=True,
        random_state=23
    ))
])

improved_clf.fit(X_train, y_train)

proba_improved = improved_clf.predict_proba(X_test)[:, 1]
yhat_improved = (proba_improved >= 0.5).astype(int)

# metrics for HistGradientBoostingClassifier
acc   = accuracy_score(y_test, yhat_improved)
auc   = roc_auc_score(y_test, proba_improved)
brier = brier_score_loss(y_test, proba_improved)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, yhat_improved, average="binary")
cm = confusion_matrix(y_test, yhat_improved)

print()
print("HistGradientBoostingClassifier Model Stats")
print(f"Accuracy    : {acc:.3f}")
print(f"ROC-AUC     : {auc:.3f}")
print(f"Brier score : {brier:.3f}")
print(f"Precision   : {prec:.3f}  Recall: {rec:.3f}  F1: {f1:.3f}")
print("Confusion matrix:\n", cm)

# results show that the HistGradientBoostingClassifier model predicts probabilities more accurately, 
# identifies almost all spenders, and has a higher accuracy, but it does make more false positives



HistGradientBoostingClassifier Model Stats
Accuracy    : 0.872
ROC-AUC     : 0.834
Brier score : 0.096
Precision   : 0.888  Recall: 0.974  F1: 0.929
Confusion matrix:
 [[ 142  398]
 [  85 3159]]


In [14]:
# 2nd model: Severity Model (cost)

# only include people who have spent money
df_copy = df[df["TOTEXP23"] > 0].copy()
X2 = df_copy[predictors]
y2 = df_copy["TOTEXP23"]


preprocess_reg = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols),
    ("num", StandardScaler(), numerical_cols),
])

# using a pipeline so that preprocessing happens before fitting the data to the GammaRegressor
base_reg = Pipeline(steps=[
    ("preprocess", preprocess_reg),
    ("model", GammaRegressor(max_iter=5000))
])

# splits dataset to train 80% and test 20%, set random_state to an integer so that it controls the randomness of how it splits the dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=23)

base_reg.fit(X2_train, y2_train)

y2_pred = base_reg.predict(X2_test)

# metrics for gamma regressor
print("GammaRegressor Model Stats")
print(f"MAE  : {mean_absolute_error(y2_test, y2_pred):,.2f}")
print(f"RMSE : {root_mean_squared_error(y2_test, y2_pred):,.2f}")
print(f"R^2  : {r2_score(y2_test, y2_pred):.3f}")


GammaRegressor Model Stats
MAE  : 9,752.80
RMSE : 20,661.76
R^2  : 0.051


In [15]:
#HistGradientBoostingRegressor
improved_reg = Pipeline(steps=[
    ("preprocess", preprocess_reg),
    ("model", HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_iter=400,
        max_depth=None,
        min_samples_leaf=20,
        l2_regularization=0.0,
        early_stopping=True,
        random_state=23
    ))
])

improved_reg.fit(X2_train, y2_train)
y2_pred = improved_reg.predict(X2_test)

# ensures that costs are nonnegative
y2_pred = np.clip(y2_pred, 0, None)

# metrics for HistGradientBoosting Regressor
mae  = mean_absolute_error(y2_test, y2_pred)
rmse = root_mean_squared_error(y2_test, y2_pred)
r2   = r2_score(y2_test, y2_pred)

print("HistGradientBoostingRegressor Model")
print(f"MAE  : {mae:,.2f}")
print(f"RMSE : {rmse:,.2f}")
print(f"R^2  : {r2:.3f}")

HistGradientBoostingRegressor Model
MAE  : 9,787.61
RMSE : 20,398.31
R^2  : 0.075


In [16]:
# Estimated Annual Cost Model
# created by combining the frequency and severity model (by multiplying them)
# EC = p x m where p is the probability of spending and m is the average cost spent

# probability of any spending
p_all = improved_clf.predict_proba(df[predictors])[:, 1]

# conditional expected cost given that spending is greater than 0
m_all = improved_reg.predict(df[predictors])
m_all = np.clip(m_all, 0, None)     # to ensure that costs aren't negative

# expected annual cost
df["EST_COST"] = p_all * m_all

# risk bands to divide population into quantiles of expected cost
# < 40th percentile: "Low", 40th-80th percentile: "Medium", > 80th percentile: "High"
low_cut, med_cut = np.quantile(df["EST_COST"], [0.40, 0.80])

def band(v):
    if v < low_cut: return "Low"
    elif v < med_cut: return "Medium"
    else: return "High"

df["RISK_BAND"] = df["EST_COST"].apply(band)

# prints the cutoff dollar values for each risk band
print("Risk band cutoffs:")
print(f"  Low < ${low_cut:,.0f}   Medium < ${med_cut:,.0f}   High ≥ ${med_cut:,.0f}")

# maps the user inputted data to the correct MEPS codes
REGION_MAP = {"northeast":1, "midwest":2, "south":3, "west":4}
SEX_MAP    = {"male":1, "female":2}
INSCOV_MAP = {"private":1, "public":2, "uninsured":3}
RACE_MAP = {"white": 1, "black": 2, "hispanic": 3, "asian": 4, "other": 5}

# converts the yes/no into bool values of 0 or 1
def to_code_bool(v):
    if isinstance(v, str):
        v = v.strip().lower()
        return 1 if v in {"1","y","yes","true","t"} else 0
    return 1 if bool(v) else 0

def make_row(age, sex, region, inscov, racethx,
             marry23x=1, povcat23=1, adsmok42=2, diabdx=False,
             cancer=False, hbp=False, asthma=False, chd=False):

    row = {
        "AGE23X": age,
        "SEX": SEX_MAP.get(str(sex).lower(), sex),
        "RACETHX": RACE_MAP.get(str(racethx).lower(), racethx),
        "REGION23": REGION_MAP.get(str(region).lower(), region),
        "MARRY23X": marry23x,     
        "POVCAT23": povcat23,       
        "INSCOV23": INSCOV_MAP.get(str(inscov).lower(), inscov),
        "CANCERDX": to_code_bool(cancer),
        "HIBPDX":   to_code_bool(hbp),
        "ASTHDX":   to_code_bool(asthma),
        "CHDDX":    to_code_bool(chd),
        "DIABDX_M18": to_code_bool(diabdx),  
        "ADSMOK42": adsmok42,     
    }

    return pd.DataFrame([row], columns=predictors)

# takes in the user's info and runs both models with it
def estimate_cost(age, sex, region, inscov, cancer=False, hbp=False, asthma=False, chd=False, racethx=None,
                  threshold=0.5):
    """Return p_i, m_i, EC, and risk band for a single user profile."""
    X_new = make_row(age, sex, region, inscov, cancer, hbp, asthma, chd, racethx)
    p = float(improved_clf.predict_proba(X_new)[:, 1])              # probability of any spending
    m = float(np.clip(improved_reg.predict(X_new)[0], 0, None))     # expected cost given spending
    ec = p * m                                                      # expected annual cost
    rb = band(ec)                                                   # risk band
    return {"p_any_spend": p, "cond_mean_cost": m, "est_cost": ec, "risk_band": rb}

Risk band cutoffs:
  Low < $4,784   Medium < $12,505   High ≥ $12,505
